# Invoking LLMs

In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv

In [2]:
load_dotenv(find_dotenv())
api_key = os.environ["OPENAI_API_KEY"]

In [3]:
content = "Was kannst Du mir über die Generali Deutschland AG sagen?"

In [4]:
client = OpenAI(
 api_key=api_key,
)

response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": content,
        }
    ],
    model="gpt-3.5-turbo",
    temperature=0.99,
)


In [5]:
print(response.choices[0].message.content)

Die Generali Deutschland AG ist ein deutsches Versicherungsunternehmen und Teil der international tätigen Generali Gruppe. Das Unternehmen wurde im Jahr 1903 gegründet und hat seinen Hauptsitz in München. Die Generali Deutschland AG bietet eine breite Palette von Versicherungsprodukten für Privat- und Geschäftskunden an, darunter Lebensversicherungen, Krankenversicherungen, Sachversicherungen und Haftpflichtversicherungen. Die Generali Deutschland AG gehört zu den größten Versicherungsunternehmen in Deutschland und beschäftigt mehrere tausend Mitarbeiter.


# LangChain Prompts

In [6]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

In [7]:
llm = ChatOpenAI(temperature=0.5, api_key=api_key)

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "Du bist ein intelligenter Assistent, der Sachbearbeiter einer Versicherungsgesellschaft in sämtlichen Fragestellungen unterstützt"),
        ("human", "Beantworte die Frage: {question}"),
    ]
)

prompt = prompt_template.format_messages(question="Was kannst Du mir über die Generali Deutschland AG sagen?")

In [8]:
respone = llm(prompt)
print(respone)

/Users/michael/Library/Caches/pypoetry/virtualenvs/langchain-experiments-SQAeIdRz-py3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


content='Die Generali Deutschland AG ist ein Teil der internationalen Generali Gruppe und gehört zu den führenden Versicherungsunternehmen in Deutschland. Die Generali Deutschland AG bietet eine breite Palette von Versicherungsprodukten für Privat- und Geschäftskunden an, darunter Lebensversicherungen, Krankenversicherungen, Haftpflichtversicherungen, Sachversicherungen und vieles mehr. Das Unternehmen legt großen Wert auf Nachhaltigkeit und soziale Verantwortung.'


# Memory

## Conversation Buffer Memory

In [9]:
from langchain.memory import ConversationBufferMemory
from langchain_openai import OpenAI
from langchain.chains import ConversationChain

See the difference between ChatOpenAI vs. OpenAI: https://stackoverflow.com/questions/76950609/what-is-the-difference-between-openai-and-chatopenai-in-langchain

In [10]:
chat = OpenAI(
    model_name="gpt-3.5-turbo-instruct", 
    temperature=0, 
    openai_api_key=api_key
)

memory = ConversationBufferMemory(size=4)
buffer_chain = ConversationChain(llm=chat, memory=memory, verbose=True)

`verbose=True` is helpful to understand the model's reasoning!

In [11]:
buffer_chain.predict(input="Beschreibe die Generali Deutschland AG")
buffer_chain.predict(input="Benutze weniger Worte für die Beschreibung")
buffer_chain.predict(input="Nutze noch weniger Worte")
buffer_chain.predict(input="Was war die erste Frage?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Beschreibe die Generali Deutschland AG
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Beschreibe die Generali Deutschland AG
AI:  Die Generali Deutschland AG ist ein deutsches Versicherungsunternehmen mit Hauptsitz in München. Sie wurde 1824 gegründet und ist Teil der internationalen Generali Group, die in über 50 Ländern tätig ist. Die Generali Deutschland AG 

' Die erste Frage war "Beschreibe die Generali Deutschland AG".'

## ConversationSummaryMemory

In [12]:
from langchain.memory import ConversationSummaryMemory

In [13]:
chat = OpenAI(
    model_name="gpt-3.5-turbo-instruct", 
    temperature=0, 
    openai_api_key=api_key
)

memory = ConversationSummaryMemory(
    llm=OpenAI(
        model_name="gpt-3.5-turbo-instruct",
        openai_api_key=api_key
    )
)
summary_chain = ConversationChain(llm=chat, memory=memory, verbose=True)

In [14]:
summary_chain.predict(input="Warum ist die Generali die beste Versicherung auf dem deutschen Markt?")
summary_chain.predict(input="Bitte erkläre nochmal spezifischer, was die Generali auszeichnet")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Warum ist die Generali die beste Versicherung auf dem deutschen Markt?
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
 The human asks why Generali is the best insurance on the German market. The AI explains that Generali has a long history of success in the insurance business, offering a wide range of insurance products such as life, health, property, and liability in

' Natürlich, gerne erkläre ich es noch genauer. Generali hat eine lange Geschichte des Erfolgs in der Versicherungsbranche und bietet eine breite Palette an Versicherungsprodukten wie Lebens-, Kranken-, Sach- und Haftpflichtversicherungen an. Darüber hinaus hat Generali eine starke finanzielle Stabilität und eine Erfolgsbilanz von prompten und zuverlässigen Zahlungen im Falle von Schadensfällen. Auch hat es eine hohe Kundenzufriedenheit und hat zahlreiche Auszeichnungen für seine Qualität und Service erhalten.'

# LCEL - LangChain Expression Language

In [15]:
from langchain_core.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser

In [16]:
model = ChatOpenAI(openai_api_key=api_key, temperature=0)

geo_prompt = PromptTemplate.from_template(
    """Du bist ein Experte in Geographie. Beschreibe die Flagge des  Landes, nach dem Du gefragt wirst. Frage: {question}. Antwort: """
)

translation_prompt = PromptTemplate.from_template(
    """Du bist ein Experte in Sprachen und Übersetzungen. Übersetze den folgenden Satz ins Englische: {answer}. Deine Übersetzung:"""
)

In [17]:
chain = (
    {"answer": geo_prompt | model | StrOutputParser()}
    | translation_prompt
    | model
    | StrOutputParser()
)

chain.invoke({"question": "Deutschland"})

'The flag of Germany consists of three horizontal stripes in the colors black, red, and gold. The colors are traditional and have a long history in Germany. The flag was first introduced in the 19th century and is now the official symbol of the country.'

# RAG - Chat with your own documents

In [24]:
from pyprojroot import here
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA

In [19]:
data = PyPDFLoader(str(here("./schadenmeldung.pdf"))).load()
print(data)

[Document(page_content='Schadensmeldung - Sturmschaden an Eigentum\nEingangszeitpunkt: 23.02.2024 21:35:19\nSehr geehrte Damen und Herren,\nich wende mich an Sie, um einen Schaden zu melden, der durch den Sturm am 22.02.2024 an\nmeinem Eigentum entstanden ist. Als Inhaber einer Hausratversicherung bei der Generali\nDeutschland Versicherung, Vertragsnummer GH567890123, möchte ich den entstandenen\nSchaden so schnell wie möglich bearbeitet wissen.\nAm besagten Tag zog ein schwerer Sturm über unseren Wohnort, der zu erheblichen Schäden an\nmeiner Wohnung im Kölner Süden, genauer in der Mainzer Straße 45, 50678 Köln, geführt hat.\nHierbei wurden Fenster beschädigt, wodurch Wasser eindringen konnte und zu Schäden an\nMöbeln, elektronischen Geräten und weiteren Hausratgegenständen führte.\nFolgende Gegenstände sind betroffen:\n- Wohnzimmertisch aus Eiche, Wasserschaden, geschätzter Zeitwert: 200EUR\n- 55 Zoll Fernsehgerät, Marke Samsung, Wasserschaden, geschätzter Wiederbeschaffungswert:\n80

In [20]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=25,
)

docs = splitter.split_documents(data)
print(docs)

[Document(page_content='Schadensmeldung - Sturmschaden an Eigentum\nEingangszeitpunkt: 23.02.2024 21:35:19', metadata={'source': '/Users/michael/Coding/langchain-experiments/schadenmeldung.pdf', 'page': 0}), Document(page_content='Sehr geehrte Damen und Herren,', metadata={'source': '/Users/michael/Coding/langchain-experiments/schadenmeldung.pdf', 'page': 0}), Document(page_content='ich wende mich an Sie, um einen Schaden zu melden, der durch den Sturm am 22.02.2024 an', metadata={'source': '/Users/michael/Coding/langchain-experiments/schadenmeldung.pdf', 'page': 0}), Document(page_content='meinem Eigentum entstanden ist. Als Inhaber einer Hausratversicherung bei der Generali', metadata={'source': '/Users/michael/Coding/langchain-experiments/schadenmeldung.pdf', 'page': 0}), Document(page_content='Deutschland Versicherung, Vertragsnummer GH567890123, möchte ich den entstandenen', metadata={'source': '/Users/michael/Coding/langchain-experiments/schadenmeldung.pdf', 'page': 0}), Document

In [21]:
embedding_function = OpenAIEmbeddings(openai_api_key=api_key)

vectordb = FAISS.from_documents(
    docs,
    embedding_function,
)

vectordb.save_local(str(here("./")))

In [25]:
qa = RetrievalQA.from_chain_type(
        llm=OpenAI(
            model_name="gpt-3.5-turbo-instruct",
            temperature=0,
            openai_api_key=api_key
        ), 
        chain_type="stuff", 
        retriever=vectordb.as_retriever()
    )

In [26]:
query = "Wann ist der Schaden entstanden?"

print(qa.run(query))

/Users/michael/Library/Caches/pypoetry/virtualenvs/langchain-experiments-SQAeIdRz-py3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


 Der Schaden ist am 23.02.2024 um 21:35:19 entstanden.


In [32]:
print(qa.run("Wann ist die Meldung eingegangen?"))
print(qa.run("Fasse den Inhalt des Dokuments in einem Satz zusammen"))
print(qa.run("Frage: An welchem Datum ist der Schaden entstaden? Hinweis: Es kann das gleiche Datum wie das Eingangsdatum der Schadenmeldung sein, es kann jedoch auch ein anderes Datum sein. Gesucht wird das Datum, an dem tatsächlich etwas beschädigt wurde."))
print(qa.run("In welche Kategorie passt die Meldung am besten? Sturm, Leitungswasser, Elementar, Einbruch, Diebstahl, Feuer, Sonstiges"))
print(qa.run("An welchem Datum war der Sturm, durch den der Schaden entstanden ist?"))



 Die Meldung ist am 23.02.2024 um 21:35:19 eingegangen.
 Das Dokument beinhaltet eine Schadensmeldung für einen Sturmschaden an Eigentum, der zu Schäden an Möbeln, elektronischen Geräten und weiteren Hausratgegenständen geführt hat.
 Das Datum, an dem der Sturmschaden an dem Eigentum entstanden ist, ist nicht angegeben. Es kann das gleiche Datum wie das Eingangsdatum der Schadenmeldung sein oder ein anderes Datum. Es ist nicht eindeutig aus dem gegebenen Kontext ersichtlich.
 Sturm
 Der Sturm war am 22.02.2024.
